In [21]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import rcParams
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn import feature_selection as fs
from sklearn import linear_model
from sklearn import metrics
rcParams['figure.figsize'] = 15, 10
import seaborn as sns
import pylab
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

import random
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import KFold
import statsmodels.formula.api as sm
import os


### Figure Size

In [60]:
rcParams['figure.figsize'] = (20.0, 15.0)

### Functions

In [59]:
def findPercentageRangesEqually(classCount,df):
    percentages = df['percentage']
    percentageArray=[]
    rangeArray=[]
    averageClassCount=len(percentages)/classCount
    averageClassCount=round(averageClassCount)
    
    for i in range(0, len(percentages)):
        percentageArray.append(percentages[i])
    
    
    percentagesArray=np.sort(percentageArray)
    
    
    for i in range(0, len(percentages)):
        if(i%averageClassCount==0):
            rangeArray.append(percentagesArray[i])
    return rangeArray

In [3]:
def makeClassDataFrame(df,classCount,command):
    rangeArray=[]
    if(command=="betweenEqual"):
        rangeArray=findPercentageRangesEqually(classCount,df)
    elif(command=="equallySliced"):
        for i in range(classCount):
            rangeArray.append(i*(1/classCount))
    classArray=[]
    ClassLibrary="ABCDEFGHIJKLMNOPRST"
    for i in range(df.shape[0]):
        for j in range(classCount):
            if(df['percentage'][i]>rangeArray[j]):
                foundClass=ClassLibrary[j]
        classArray.append(foundClass)
    df['ClassLabel']=pd.Series(classArray)
    df2=df.drop(columns=['percentage'])
    return df2

In [4]:
def FeatureSelection(df):
    X = pd.DataFrame(df.iloc[:,:-1])
    y = pd.DataFrame(df.iloc[:,-1])
    y = pd.get_dummies(y)
    model = ExtraTreesClassifier(random_state=1)
    model.fit(X, y)
    # display the relative importance of each attribute
    features = list(X.columns)

    y_pos = np.arange(len(features))

    d={}
    c=0
    for i in features:
        d[i]=model.feature_importances_[c]
        c=c+1
    newd = sorted(d.items(), key=lambda kv: kv[1])
    dd={}
    for i in range(0,len(features)):
        dd[newd[i][0]]=newd[i][1]
    plt.barh(range(len(dd)), list(dd.values()), align='center')
    plt.yticks(range(len(dd)), list(dd.keys()))
    plt.show()
    print(dd)
    return dd


In [5]:
def FeatureSelectionRegression(df):
    import pandas as pd
    X = pd.DataFrame(df.iloc[:,:-1])
    y = pd.DataFrame(df.iloc[:,-1])
    features = list(X.columns)
    f_and_p_test=list()
    f_test, p_test=fs.f_regression(X, y, center=True)
    
    fd={}
    c=0
    for i in features:
        fd[i]=f_test[c]
        c=c+1
    newfd = sorted(fd.items(), key=lambda kv: kv[1])
    fdd={}
    for i in range(0,len(features)):
        fdd[newfd[i][0]]=newfd[i][1]
    plt.barh(range(len(fdd)), list(fdd.values()), align='center')
    plt.yticks(range(len(fdd)), list(fdd.keys()))
    plt.show()
    
    pd={}
    t=0
    for i in features:
        pd[i]=f_test[t]
        t=t+1
    newpd = sorted(pd.items(), key=lambda kv: kv[1])
    pdd={}
    for j in range(0,len(features)):
        pdd[newpd[j][0]]=newpd[j][1]
    plt.barh(range(len(pdd)), list(pdd.values()), align='center')
    plt.yticks(range(len(pdd)), list(pdd.keys()))
    plt.show()
    
    return fdd,pdd
    

In [6]:
def sortFeatures(keyList,corrArray):
    
    sortedArray=[]
    for i in corrArray:
        sortedSubArray=[]
        for j in len(i):
            for t in keyList:
                if(i[j]==t):
                    sortedSubArrray.append(t)
        sortedArray.append(sortedSubArray)

In [27]:
def TreeMethod(methodName, estimator):
    #you can add other tree classifiers
    if(methodName=="AdaBoost"):
        clf=AdaBoostClassifier(n_estimators=estimator)
    elif(methodName=="ExtraTrees"):
        clf=ExtraTreesClassifier(n_estimators=estimator, criterion="gini", max_depth=None, bootstrap=True)
         

In [32]:
def TreeClassifier(classCount,csvName,command,methodName):
    file = open(csvName+'_'+methodName+'_'+command+'_C_'+str(classCount)+'.txt')
    df2 = pd.read_csv(csvName)
    df2 = df2.dropna()
    df2=df2.reset_index(drop=True)
    df=makeClassDataFrame(df2,classCount,command)
    dd=FeatureSelection(df)
    keyList=list(dd.keys())
    lines_of_text=[]
    print("ClassCount="+str(classCount)+"\n")
    lines_of_text.append("ClassCount="+str(classCount)+"\n")
    featuresLen = len(list(df2.columns))-3
    for i in range(0,featuresLen):
        droppedColumn=keyList[i]
        df=df.drop(columns=[str(droppedColumn)])
        X = pd.DataFrame(df.iloc[:,:-1])
        X = (X - X.mean()) / (X.max() - X.min())
        y = pd.DataFrame(df.iloc[:,-1])
        y = pd.get_dummies(y)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
        ### convert one hot to numbers
        y_train = np.argmax(y_train.values, axis=-1)

        y_true = np.argmax(y_test.values, axis=-1)
        maxAccuracy=0
        lineString=""
        for j in range(1,51):#change range
            estimator=j*100
            clf = TreeMethod(methodName,estimator)
            clf.fit(X_train.values, y_train)
            y_pred = clf.predict(X_test.values)
            accuracy=accuracy_score(y_true, y_pred)
            if(accuracy>maxAccuracy):
                maxAccuracy=accuracy
                lineString="DroppedColumn:"+droppedColumn+" Estimator:" +str(estimator)+" Accuracy:"+str(accuracy)+"\n"
                
        print(lineString)
        file.write(lineString)
        print("ClassCount:"+ str(classCount)+ " Finished:"+droppedColumn)

    file.close() 

In [33]:
def SVM(classCount,csvName,command):
    file = open(csvName+'_SVM_'+command+'_C_'+str(classCount)+'.txt','w')
    df2 = pd.read_csv(csvName)
    df2 = df2.dropna()
    df2=df2.reset_index(drop=True)
    df=makeClassDataFrame(df2,classCount,command)
    dd=FeatureSelection(df)
    keyList=list(dd.keys())
    lines_of_text=[]
    print("ClassCount="+str(classCount)+"\n")
    lines_of_text.append("ClassCount="+str(classCount)+"\n")
    featuresLen = len(list(df2.columns))-3
    for i in range(0,featuresLen):
        droppedColumn=keyList[i]
        df=df.drop(columns=[str(droppedColumn)])
        X = pd.DataFrame(df.iloc[:,:-1])
        X = (X - X.mean()) / (X.max() - X.min())
        y = pd.DataFrame(df.iloc[:,-1])
        y = pd.get_dummies(y)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
        ### convert one hot to numbers
        y_train = np.argmax(y_train.values, axis=-1)

        y_true = np.argmax(y_test.values, axis=-1)
        maxAccuracy=0
        lineString=""
        for j in range(1,51):#change range
            Cnew=j*0.01
            clf = SVC(C = Cnew, kernel='rbf', degree=3, )
            clf.fit(X_train.values, y_train)

            y_pred = clf.predict(X_test.values)

            accuracy=accuracy_score(y_true, y_pred)
            
            if(accuracy>maxAccuracy):
                maxAccuracy=accuracy
                lineString="DroppedColumn:"+droppedColumn+" C:" +str(Cnew)+" Accuracy:"+str(accuracy)+"\n"
                
        print(lineString)
        file.write(lineString)
        print("ClassCount:"+ str(classCount)+ " Finished:"+droppedColumn)

    file.close() 

In [34]:
def SGDRegressor(csvName):
    file = open('SGDRegressorTest'+'.txt','w')
    df = pd.read_csv(csvName)
    df = df.dropna()
    df=df.reset_index(drop=True)
    fdd,pdd=FeatureSelectionRegression(df)
    keyList=list(fdd.keys())
    lines_of_text=[]
    print("F Test Commencing\n")
    featuresLen = len(list(df.columns))-2
    for i in range(0,featuresLen):
        droppedColumn=keyList[i]
        df=df.drop(columns=[str(droppedColumn)])
        X = pd.DataFrame(df.iloc[:,:-1])
        X = (X - X.mean()) / (X.max() - X.min())
        y = pd.DataFrame(df.iloc[:,-1])
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
        ### convert one hot to numbers
        y_true=y_test
        maxAccuracy=0
        lineString=""
        for j in range(1,51):#change range
            estimator=j*100
            clf = linear_model.SGDRegressor()
            clf.fit(X_train.values, y_train)
            y_pred = clf.predict(X_test.values)
            accuracy=metrics.r2_score(y_true, y_pred)
            
            if(accuracy>maxAccuracy):
                maxAccuracy=accuracy
                lineString="DroppedColumn:"+droppedColumn+" Test:F MSE:"+str(accuracy)+"\n"
                
        print(lineString)
        file.write(lineString)
    df = pd.read_csv(csvName)
    df = df.dropna()
    keyList=list(pdd.keys())
    lines_of_text=[]
    print("P Test Commencing\n")
    for i in range(0,featuresLen):
        droppedColumn=keyList[i]
        df=df.drop(columns=[str(droppedColumn)])
        X = pd.DataFrame(df.iloc[:,:-1])
        X = (X - X.mean()) / (X.max() - X.min())
        y = pd.DataFrame(df.iloc[:,-1])
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
        ### convert one hot to numbers
        y_true=y_test
        maxAccuracy=0
        lineString=""
        for j in range(1,51):#change range
            estimator=j*100
            clf = linear_model.SGDRegressor()
            clf.fit(X_train.values, y_train)
            y_pred = clf.predict(X_test.values)
            accuracy=metrics.r2_score(y_true, y_pred)
            
            if(accuracy>maxAccuracy):
                maxAccuracy=accuracy
                lineString="DroppedColumn:"+droppedColumn+" Test:P MSE:"+str(accuracy)+"\n"
                
        print(lineString)
        file.write(lineString)

    file.close() 

In [35]:
def Elastic(csvName):
    file = open('ElasticTest'+csvName[4:]+'.txt','w+')
    df = pd.read_csv(csvName)
    df=df.dropna()
    df=df.reset_index(drop=True)
    fdd,pdd=FeatureSelectionRegression(df)
    keyList=list(fdd.keys())
    lines_of_text=[]
    print("F Test Commencing\n")
    featuresLen = len(list(df.columns))-2
    for i in range(0,featuresLen):
        droppedColumn=keyList[i]
        df=df.drop(columns=[str(droppedColumn)])
        X = pd.DataFrame(df2.iloc[:,:-1])
        X = (X - X.mean()) / (X.max() - X.min())
        y = pd.DataFrame(df2.iloc[:,-1])
        
        sc_x = StandardScaler()
        sc_y = StandardScaler()
        # Scale x and y (two scale objects)
        X = sc_x.fit_transform(X)
        y = sc_y.fit_transform(y)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
        ### convert one hot to numbers
        y_true=y_test
        maxAccuracy=1
        lineString=""
        for j in range(1,100):#change range
            for t in range(1,100):
                alpha1=j*0.001
                l1=t*0.01
                clf =linear_model.ElasticNet(alpha=alpha1, l1_ratio=l1, max_iter=10000, positive=False, precompute=False, random_state=0, selection='cyclic', tol=0.0001, warm_start=False)
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)
                accuracy=metrics.r2_score(y_true, y_pred)
                if(accuracy<maxAccuracy):
                    maxAccuracy=accuracy
                    lineString="DroppedColumn:"+droppedColumn+" Alpha:"+str(alpha1)+" L1 ratio:"+str(l1) +" Test:F MSE:"+str(accuracy)+"\n"
                
        print(lineString)
        file.write(lineString)
    df = pd.read_csv(csvName)
    df = df.dropna()
    keyList=list(pdd.keys())
    lines_of_text=[]
    print("P Test Commencing\n")
    for i in range(0,featuresLen):
        droppedColumn=keyList[i]
        df=df.drop(columns=[str(droppedColumn)])
        X = pd.DataFrame(df.iloc[:,:-1])
        X = (X - X.mean()) / (X.max() - X.min())
        y = pd.DataFrame(df.iloc[:,-1])
        
        sc_x = StandardScaler()
        sc_y = StandardScaler()
        # Scale x and y (two scale objects)
        X = sc_x.fit_transform(X)
        y = sc_y.fit_transform(y)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
        ### convert one hot to numbers
        y_true=y_test
        maxAccuracy=1
        lineString=""
        for j in range(1,100):#change range
            for t in range(1,100):
                alpha1=j*0.001
                l1=t*0.01
                clf =linear_model.ElasticNet(alpha=alpha1, l1_ratio=l1, max_iter=10000, positive=False, precompute=False, random_state=0, selection='cyclic', tol=0.0001, warm_start=False)
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)
                accuracy=metrics.r2_score(y_true, y_pred)
                if(accuracy<maxAccuracy):
                    maxAccuracy=accuracy
                    lineString="DroppedColumn:"+droppedColumn+" Alpha:"+str(alpha1)+" L1 ratio:"+str(l1) +" Test:P MSE:"+str(accuracy)+"\n"
                
        print(lineString)
        file.write(lineString)

    file.close() 

In [42]:
def NeuralNet(classCount,csvName,command,layerCount):
    hlayer = list()
    alphalist = [1e-3, 1e-5, 1e-8, 1e-10] #you can change alpha parameters
    file = open(csvName+'_NeuralNet_'+command+'_C_'+str(classCount)+'_L_'+str(layerCount)+'.txt')
    
    df2 = pd.read_csv(csvName)
    df2 = df2.dropna()
    df2=df2.reset_index(drop=True)
    df=makeClassDataFrame(df2,classCount,command)
    dd=FeatureSelection(df)
    keyList=list(dd.keys())
    lines_of_text=[]
    print("ClassCount="+str(classCount)+"\n")
    lines_of_text.append("ClassCount="+str(classCount)+"\n")
    
    
    if(layerCount==2):
        for i in range(1,11):
            for j in range(1,11): 
                tup = (random.randint(1,21)*25,random.randint(1,21)*5)           
                hlayer.append(tup)
  
    
    if(layerCount==3):
        for i in range(1,11):
            for j in range(1,11): 
                tup = (random.randint(1,21)*25,random.randint(1,21)*5,random.randint(1,21)*5)           
                hlayer.append(tup)
    
    if(layerCount==4):
        for i in range(1,11):
            for j in range(1,11): 
                tup = (random.randint(1,21)*25,random.randint(1,21)*5,random.randint(1,21)*5,random.randint(1,21)*25)           
                hlayer.append(tup)      
    
    if(layerCount==5):
        for i in range(1,11):
            for j in range(1,11): 
                tup = (random.randint(1,21)*25,random.randint(1,21)*5,random.randint(1,21)*5,random.randint(1,21)*25,random.randint(1,21)*25)           
                hlayer.append(tup)
    
    else:
        print("Please enter a valid layer count")
        return
        
    X = pd.DataFrame(df.iloc[:,:-1])
    X = (X - X.mean()) / (X.max() - X.min())
    y = pd.DataFrame(df.iloc[:,-1])
    y = pd.get_dummies(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
    ### convert one hot to numbers
    y_train = np.argmax(y_train.values, axis=-1)
    y_true = np.argmax(y_test.values, axis=-1)
    maxAccuracy=0
    lineString=""
    
    
    
    
    for j in range(0,len(hlayer)):#change range
            for i in range(0, len(alphalist)):
                clf = MLPClassifier(solver='lbfgs', alpha=alphalist[i], hidden_layer_sizes=hlayer[j], random_state=1)
                clf.fit(X_train.values, y_train)
                y_pred = clf.predict(X_test.values)
                accuracy=accuracy_score(y_true, y_pred)
                if(accuracy>maxAccuracy):
                    maxAccuracy=accuracy
                    
                    lineString="DroppedColumn:NoDroppedColumn HH:" +str(hlayer[j]) + " Alpha:" + str(alphalist[i])+" Accuracy:"+str(accuracy)+"\n"
    
    
    print(lineString)
    file.write(lineString)
    print("ClassCount:"+ str(classCount)+ " Finished: NoDroppedColumn")
    
    for i in range(0,22):
        droppedColumn=keyList[i]
        df=df.drop(columns=[str(droppedColumn)])
        X = pd.DataFrame(df.iloc[:,:-1])
        X = (X - X.mean()) / (X.max() - X.min())
        y = pd.DataFrame(df.iloc[:,-1])
        y = pd.get_dummies(y)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
        ### convert one hot to numbers
        y_train = np.argmax(y_train.values, axis=-1)

        y_true = np.argmax(y_test.values, axis=-1)
        maxAccuracy=0
        lineString=""
        
        for j in range(0,len(hlayer)):#change range
            for i in range(0, len(alphalist)):
                clf = MLPClassifier(solver='lbfgs', alpha=alphalist[i], hidden_layer_sizes=hlayer[j], random_state=1)
                clf.fit(X_train.values, y_train)
                y_pred = clf.predict(X_test.values)
                accuracy=accuracy_score(y_true, y_pred)
                if(accuracy>maxAccuracy):
                    maxAccuracy=accuracy
                    lineString="DroppedColumn:"+droppedColumn+" HH:" +str(hlayer[j]) + " Alpha:" + str(alphalist[i])+" Accuracy:"+str(accuracy)+"\n"


        print(lineString)
        file.write(lineString)
        print("ClassCount:"+ str(classCount)+ " Finished:"+droppedColumn)

    file.close()

In [43]:
def heatMap(csvName):
    df = pd.read_csv(csvName)
    df=df.dropna()
    sns.heatmap(df.corr(), annot=True, fmt=".2f")


In [44]:
def printDataHistograms(csvName):
    df = pd.read_csv(csvName)
    df=df.dropna()
    for column in df.columns:
        ax=sns.distplot(df[column])
        plt.pause(0.0001)

In [45]:
def scatter_plot_with_correlation_line(x, y, graph_filepath):
    '''
    http://stackoverflow.com/a/34571821/395857
    x does not have to be ordered.
    '''
    # Scatter plot
    plt.scatter(x, y)

    # Add correlation line
    axes = plt.gca()
    m, b = np.polyfit(x, y, 1)
    X_plot = np.linspace(axes.get_xlim()[0],axes.get_xlim()[1],100)
    plt.plot(X_plot, m*X_plot + b, '-')

    # Save figure
    plt.savefig(graph_filepath, dpi=300, format='png', bbox_inches='tight')

In [46]:
def reg_m(y, x):
    ones = np.ones(len(x[0]))
    X = sm.add_constant(np.column_stack((x[0], ones)))
    for ele in x[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
    results = sm.OLS(y, X).fit()
    return results

In [47]:
def ElasticExperiment(csvName):

    df2 = pd.read_csv(csvName)
    df2=df2.dropna()
    df2=df2.reset_index(drop=True)



    X = pd.DataFrame(df2.iloc[:,:-1])
    y = pd.DataFrame(df2.iloc[:,-1])


    sc_x = StandardScaler()
    sc_y = StandardScaler()
    # Scale x and y (two scale objects)
    X = sc_x.fit_transform(X)
    y = sc_y.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
            ### convert one hot to numbers
    y_true=y_test






    """"clf = AdaBoostClassifier(n_estimators=100)
    clf.fit(X_train.values, y_train)
    y_pred = clf.predict(X_test.values)
    accuracy=accuracy_score(y_true, y_pred)"""

    clf = linear_model.ElasticNet(alpha=0.001, l1_ratio=0.01,
          max_iter=100000, positive=False, precompute=False,
          random_state=0, selection='cyclic', tol=0.0001, warm_start=False)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy=metrics.r2_score(y_true, y_pred)

    print(accuracy)
    axis.sort()
    plt.scatter(axis,y_test,  color='black')
    plt.scatter(axis,y_pred,  color='blue')

    plt.xticks(())
    plt.yticks(())



    for i in range(0,len(axis)):
        plt.plot([axis[i],axis[i]], [y_pred[i],y_test[i]], 'r-')
    



In [51]:
def saveConfusionMatricesNeuralNet(csvName, path,command,classCount,layerCount):
    
    classes = [['A', 'B'],['A', 'B', 'C'], ['A', 'B', 'C', 'D'],['A', 'B', 'C', 'D','E'],['A', 'B', 'C', 'D','E','F']]
    files=[]
    for i in range(classCount[0],classCount[1]+1):
        files.append(path+'/'+csvName+'_NeuralNet_'+command+'_C_'+str(i)+'_L_'+str(layerCount)+'.txt')
    for i in range(len(files)):
        acc_list = list()
        metric_list = list()
        hiddenlayer_list = list()
        alphaList=list()
        class_val = files[i][len(files[i])-5]
        print(class_val)
        with open(files[i]) as file:
            for line in file: 
                if not line.strip(): continue
                h_list=list()
                colIndex1=line.find(':')+1
                colIndex2=line.find("HH")-1
                droppedColumn=line[colIndex1:colIndex2]
                
                hlayerIndex1=line.find(':',colIndex2)+1
                hlayerIndex2=line.find(')',hlayerIndex1)
                h_layer=line[hlayerIndex1+1:hlayerIndex2]
                h_layer=str(h_layer)
                h_layer=h_layer.replace(",","")
                h_layer=h_layer.split()
                real=()
                
                
                for b in range(0,int(layerCount)):
                    real=real+(int(h_layer[b]),)
                
                
                
                print(real)
                AlphaIndex1=line.find(':',hlayerIndex2)+1
                AlphaIndex2=line.find(' ',AlphaIndex1)
                alpha=float(line[AlphaIndex1:AlphaIndex2])
                
                
                
                AccIndex1=line.find(':',AlphaIndex2)+1
                AccIndex2=line.find(' ',line.find('\n'))
                accuracy=float(line[AccIndex1:AccIndex2])
                
                
                
                
                
                
                
                alphaList.append(alpha)
                hiddenlayer_list.append(real)
                
                
                
                metric_list.append(droppedColumn)
                acc_list.append(accuracy)
        #print(acc_list)  
       # print(metric_list)
        max_acc = max(acc_list)
        index = acc_list.index(max_acc)
        print(metric_list)
        #print(hiddenlayer_list[index])

        plt.barh(np.arange(len(metric_list)), acc_list)
        plt.yticks(np.arange(len(metric_list)), metric_list)
        plt.xlabel('Accuracy')
        plt.title('Neural Net ' + str(class_val)+ ' Classes '+str(layerCount)+' Hidden Layer')
        plt.savefig(csvName+'_NeuralNet_AccStack_C_'+str(class_val)+'_L_'+layerCount+'.png')
        plt.show()

        
        df2 = pd.read_csv(csvName)
        df2 = df2.dropna()
        df2=df2.reset_index(drop=True)
        df=makeClassDataFrame(df2,int(class_val),"equallySliced")
        i_class_val = int(class_val)


        for i in range(0,index+1):
            if(metric_list[i]!="NoDroppedColumn"):
                df=df.drop(columns=metric_list[i])
            X = pd.DataFrame(df.iloc[:,:-1])
            X = (X - X.mean()) / (X.max() - X.min())
            y = pd.DataFrame(df.iloc[:,-1])
            y = pd.get_dummies(y)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
                ### convert one hot to numbers
            y_train = np.argmax(y_train.values, axis=-1)

            y_true = np.argmax(y_test.values, axis=-1)


            clf = MLPClassifier(solver='lbfgs', alpha=alphaList[i], hidden_layer_sizes=(hiddenlayer_list[i]), random_state=1)
            clf.fit(X_train.values, y_train)
            y_pred = clf.predict(X_test.values)
            accuracy=accuracy_score(y_true, y_pred)

            print(accuracy)

            cnf_matrix = confusion_matrix(y_true, y_pred)
            cmap = plt.get_cmap('Blues')
            cnf_matrix = confusion_matrix(y_true, y_pred)
            plt.figure(figsize=(8, 6))
            plt.imshow(cnf_matrix, interpolation='nearest', cmap=cmap)
            print(cnf_matrix)
            tick_marks = np.arange(len(classes[i_class_val-2]))
            plt.xticks(tick_marks, classes[i_class_val-2])
            plt.yticks(tick_marks, classes[i_class_val-2])
            #plt.matshow(cnf_matrix)
            plt.title('Confusion matrix for class size ' + str(class_val) + ' HiddenLayer = ' + str(hiddenlayer_list[i]) +' LR: ' + str(alphaList[i]))
            thresh = cnf_matrix.max() / 2
            for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
                    plt.text(j, i, format(cnf_matrix[i, j]),
                             horizontalalignment="center",
                             color="white" if cnf_matrix[i, j] > thresh else "black")
            plt.colorbar()
            plt.ylabel('True label')
            plt.xlabel('Predicted label')
            
            plt.savefig(csvName+'_NeuralNet_CM_C_'+str(class_val)+'_L_'+str(layerCount)+'.png')
            plt.show()

In [52]:
def saveConfusionMatricesTrees(csvName,path,treeMethodName,command,classCount):
    classes = [['A', 'B'],['A', 'B', 'C'], ['A', 'B', 'C', 'D'],['A', 'B', 'C', 'D','E'],['A', 'B', 'C', 'D','E','F']]

    files=[]
    for i in range(classCount[0],classCount[1]+1):
        files.append(path+'/'+csvName+'_'+treeMethodName+'_'+command+'_C_'+str(i)+'.txt')
    for i in range(len(files)):
        acc_list = list()
        metric_list = list()
        hiddenlayer_list = list()
        est_list = list()
        filename_split = files[i].split("/")
        class_val = filename_split[1][14:15]
        with open(files[i]) as file:

            for line in file: 
                if not line.strip(): continue       
                line = line.replace('\n','')
                splitted = line.split(":") 
                est = splitted[2]
                est2 = est.replace('Accuracy','')
                estimator = est2.strip()
                est_list.append(estimator)
                split_val = splitted[1].replace('Estimator', '')
                split_val2 = split_val.strip()
                metric_list.append(split_val2)
                acc_list.append(float(splitted[3])*100)
        #print(acc_list)  
        #print(metric_list)
        #print(est_list)
        max_acc = max(acc_list)
        print(max_acc)
        index = acc_list.index(max_acc)
        print(metric_list[index])
        print(est_list[index])

        plt.barh(np.arange(len(metric_list)), acc_list)
        plt.yticks(np.arange(len(metric_list)), metric_list)
        plt.xlabel('Accuracy')
        plt.title(threeMethodName+' '+ str(class_val) + ' Classes')
        
        plt.savefig(csvName+'_'+treeMethodName+'_AccStack_C'+str(class_val)+'.png')
        plt.show()

        
        df2 = pd.read_csv(csvName)
        df2 = df2.dropna()
        df2=df2.reset_index(drop=True)
        df=makeClassDataFrame(df2,int(class_val),command)



        for i in range(0,index+1):

            df=df.drop(columns=metric_list[i])
            X = pd.DataFrame(df.iloc[:,:-1])
            X = (X - X.mean()) / (X.max() - X.min())
            y = pd.DataFrame(df.iloc[:,-1])
            y = pd.get_dummies(y)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
            ### convert one hot to numbers
            y_train = np.argmax(y_train.values, axis=-1)

            y_true = np.argmax(y_test.values, axis=-1)


        clf = TreeMethod(n_estimators=estimator)
        clf.fit(X_train.values, y_train)
        y_pred = clf.predict(X_test.values)
        accuracy=accuracy_score(y_true, y_pred)

        print(accuracy)

        cnf_matrix = confusion_matrix(y_true, y_pred)
        cmap = plt.get_cmap('Blues')
        cnf_matrix = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(8, 6))
        plt.imshow(cnf_matrix, interpolation='nearest', cmap=cmap)
        print(cnf_matrix)
        tick_marks = np.arange(len(classes[int(class_val)-2]))
        plt.xticks(tick_marks, classes[int(class_val)-2])
        plt.yticks(tick_marks, classes[int(class_val)-2])
        #plt.matshow(cnf_matrix)
        plt.title('Confusion matrix for '+treeMethodName+' class size ' + str(class_val) +  ' Estimator ' + str(est_list[index]))
        thresh = cnf_matrix.max() / 2
        for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
                plt.text(j, i, format(cnf_matrix[i, j]),
                         horizontalalignment="center",
                         color="white" if cnf_matrix[i, j] > thresh else "black")
        plt.colorbar()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.savefig(csvName+'_'+treeMethodName+'_CM_C_'+str(class_val)+'.png')
        plt.show()

In [53]:
def saveConfusionMatricesSVM(csvName,path,command,classCount):
    classes = [['A', 'B'],['A', 'B', 'C'], ['A', 'B', 'C', 'D'],['A', 'B', 'C', 'D','E'],['A', 'B', 'C', 'D','E','F']]
    
    
    files=[]
    for i in range(classCount[0],classCount[1]+1):
        files.append(path+'/'+csvName+'_SVM_'+command+'_C_'+str(i)+'.txt')
    classes = [['A', 'B'],['A', 'B', 'C'], ['A', 'B', 'C', 'D'],['A', 'B', 'C', 'D','E'],['A', 'B', 'C', 'D','E','F']]

    for i in range(len(files)):
        acc_list = list()
        metric_list = list()
        hiddenlayer_list = list()
        est_list = list()
        filename_split = files[i].split("/")
        class_val = filename_split[1][3:4]
        print(class_val)
        with open(files[i]) as file:


            for line in file: 
                if not line.strip(): continue       
                line = line.replace('\n','')
                splitted = line.split(":") 
                est = splitted[2]
                est2 = est.replace('Accuracy','')
                estimator = est2.strip()
                est_list.append(estimator)
                split_val = splitted[1][:-1]
                split_val2 = split_val.strip()
                metric_list.append(split_val2)
                acc_list.append(float(splitted[3])*100)
        #print(acc_list)  
        #print(metric_list)
        #print(est_list)
        max_acc = max(acc_list)
        print(max_acc)
        index = acc_list.index(max_acc)
        print(metric_list[index])
        print(est_list[index])

        plt.barh(np.arange(len(metric_list)), acc_list)
        plt.yticks(np.arange(len(metric_list)), metric_list)
        plt.xlabel('Accuracy')
        plt.title('SVM ' + str(class_val) + ' Classes')
        plt.savefig(csvName+'_SVM_AccStack_C'+str(class_val)+'.png')
        plt.show()

        df2 = pd.read_csv(csvName)
        df2 = df2.dropna()
        df2=df2.reset_index(drop=True)
        df=makeClassDataFrame(df2,int(class_val),command)



        for i in range(0,index+1):

            df=df.drop(columns=metric_list[i])
            X = pd.DataFrame(df.iloc[:,:-1])
            X = (X - X.mean()) / (X.max() - X.min())
            y = pd.DataFrame(df.iloc[:,-1])
            y = pd.get_dummies(y)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
            ### convert one hot to numbers
            y_train = np.argmax(y_train.values, axis=-1)

            y_true = np.argmax(y_test.values, axis=-1)



        clf = SVC(C = float(est_list[index]), kernel='rbf', degree=3, )
        clf.fit(X_train.values, y_train)
        clf.fit(X_train.values, y_train)

        y_pred = clf.predict(X_test.values)
        accuracy=accuracy_score(y_true, y_pred)

        print(accuracy)

        cnf_matrix = confusion_matrix(y_true, y_pred)
        cmap = plt.get_cmap('Blues')
        cnf_matrix = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(8, 6))
        plt.imshow(cnf_matrix, interpolation='nearest', cmap=cmap)
        print(cnf_matrix)
        tick_marks = np.arange(len(classes[int(class_val)-2]))
        plt.xticks(tick_marks, classes[int(class_val)-2])
        plt.yticks(tick_marks, classes[int(class_val)-2])
        #plt.matshow(cnf_matrix)
        plt.title('Confusion matrix for SVM class size:' +  str(class_val) +  ' C_val:'  + str(est_list[index])+' Accuracy:'+str(accuracy))
        thresh = cnf_matrix.max() / 2
        for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
                plt.text(j, i, format(cnf_matrix[i, j]),
                         horizontalalignment="center",
                         color="white" if cnf_matrix[i, j] > thresh else "black")
        plt.colorbar()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.savefig(csvName+'_SVM_CM_C_'+str(class_val)+'.png')
        plt.show()

### Machine Learning Example

In [ ]:
#your target value must be always at the end, and must be called percentage, you can change default name in the function
csvName="chart_closure_lang_time_cyclomatic.csv"

#distribute your classes according to your command (useEquallySliced for default)
command1="equallySliced"
command2="betweenEqual"

#look heatmap to see correlated features and print histograms of each data
heatMap(csvName)
printDataHistograms(csvName)

#tree classifier names
adaboost="AdaBoost"
extratrees="ExtraTrees"

classSizeRange1=2
classSizeRange2=6
layerCount=2
for i in range(classSizeRange1,classSizeRange2+1):
    print("Class:"+str(i)+"\n")
    #AdaBoost(i,csvName,command1)
    #TreeClassifier(i,csvName,command1,adaboost)
    #TreeClassifier(i,csvName,command1,extratrees)
    #SVM(i,csvName,command1)
    #NeuralNet(i,csvName,layerCount)
Elastic(csvName)




### Confusion Matrix Generator Example

In [57]:


classSizeRange1=2
classSizeRange2=6
classCountRange = [None] * 2
#range of  your class counts for instance if you have text files for 2, 3 and 4 classes please
#initiliaze classSizeRange1=2 and classSizeRange2=4

classCountRange[0]=classSizeRange1
classCountRange[1]=classSizeRange2
#change the path according to your folders
defaultLayerPath=os.getcwd()

saveConfusionMatricesTrees(csvName,path,treeMethodName,command,classCountRange):
saveConfusionMatricesNeuralNet(csvName, defaultLayerPath,command1,classCountRange,layerCount)
saveConfusionMatricesSVM(csvName,path,command,classCountRange):